1. get geo index for each county
2. write function to get DataFrame of 1. city 2. type 3. page token (optional)
3. write function to get 60 places DataFrame
4. get multiple city / types
5. join them
6. profit!

In [1]:
# Dependencies
import requests
import json
import time
import pandas as pd
import numpy as np
import pprint as pprint
# Google API Key
from config import gkey

def to_lat_lng(json):
  return '{0}, {1}'.format(json["lat"], json['lng'])

def get_cities_df(cities):
    points = []
    north_easts = []
    south_wests = []
    
    for target_city in cities: 
        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(target_city, gkey)
        # Run a request to endpoint and convert result to json
        response = requests.get(target_url).json()
        geometry = response["results"][0]["geometry"]

        # Extract latitude and longitude.
        points.append(to_lat_lng(geometry["location"]))
        north_easts.append(to_lat_lng(geometry["bounds"]["northeast"]))
        south_wests.append(to_lat_lng(geometry["bounds"]["southwest"]))

    return pd.DataFrame({
        "City": cities,
        "Point": points,
        "Northeast": north_easts,
        "Southwest": south_wests,
    }) 

# cities = ["Palo Alto, California"]
cities = ["San Francisco, California", "Redwood City, California", "Berkeley, California", "Palo Alto, California", "Mountain View, California", "Sunnyvale, California", "Cupertino, California", "Santa Clara, California", "San Jose, California", "Fremont, California"]
# cities_df = get_cities_df(cities)
# cities_df.to_csv('cities.csv')



In [2]:


def get_response(url, params):
    response = requests.get(url, params=params).json()
    if response['status'] == 'OK':
        return response
    else:
        print('Not OK response: ' + response['status'])
        return None

# Returns DataFrame
def request_places(city, location, type, gkey):    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # rankby:distance will result INVALID_REQUSET
    params = {
        "location": location,
        "radius": 10000, # hard-code distance
        "type": type,
        "key": gkey,
    }

    response = get_response(base_url, params)
    results= response['results']
    count = 1
    
    # If there is no page token, return result, otherwise concat result with next page.
    while (count < 3 and 'next_page_token' in response):
        count = count + 1
        token = response['next_page_token']
#         print(f'Getting next page: {count}, token: {token}')
        
        # Sleep a while so that token can be used.
        time.sleep(3)
        params = {
            "location": location,
            "radius": 10000, # hard-code distance
            "type": type,
            "key": gkey,
            "page_token": response['next_page_token'],
        }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = get_response(base_url, params)
        results.extend(response['results'])
    
    print(f'Total page count: {count}')
    return results

def to_data_frame(city, type, results):
    cities = []
    types = []
    place_ids = []
    names = []
    poi = []
    vicnities = []
    locations = []
    
    for result in results:
        # Skip locality place, e.g. Palo Alto.
        if ('locality' in result['types']):
            continue

        cities.append(city)
        types.append(type)
        place_ids.append(result['place_id'])
        names.append(result['name'])
        poi.append(result['types'])
        vicnities.append(result['vicinity'])
        locations.append(to_lat_lng(result['geometry']['location']))
    
    return pd.DataFrame({
        'City': cities,
        'Type': types,
        'Place ID': place_ids,
        'Name': names,
        'Point of interests': poi,
        'Address': vicnities,
        'LatLng': locations,
    })

def get_places_data_frame(city, location, type, gkey):
    print(f'Getting plcaes, city: "{city}", location: "{location}", type: "{type}"');
    results = request_places(city, location, type, gkey)
    return to_data_frame(city, type, results) 

In [3]:
cities_df = pd.read_csv("cities.csv")
type = 'restaurant'

def get_all_places_data_frame(type, gkey):
    frames = [];
    for index, row in cities_df.iterrows():
        frames.append(get_places_data_frame(row['City'], row['Point'], type, gkey))
    return pd.concat(frames)

place_df = get_all_places_data_frame(type, gkey)
place_df.to_csv(type + '.csv')
place_df





Getting plcaes, city: San Francisco, California, location: 37.7749295, -122.4194155, type: restaurant
Total page count: 3
Getting plcaes, city: Redwood City, California, location: 37.48521520000001, -122.2363548, type: restaurant
Total page count: 3
Getting plcaes, city: Berkeley, California, location: 37.8715926, -122.272747, type: restaurant
Total page count: 3
Getting plcaes, city: Palo Alto, California, location: 37.4418834, -122.1430195, type: restaurant
Total page count: 3
Getting plcaes, city: Mountain View, California, location: 37.3860517, -122.0838511, type: restaurant
Total page count: 3
Getting plcaes, city: Sunnyvale, California, location: 37.36883, -122.0363496, type: restaurant
Total page count: 3
Getting plcaes, city: Cupertino, California, location: 37.3229978, -122.0321823, type: restaurant
Total page count: 3
Getting plcaes, city: Santa Clara, California, location: 37.3541079, -121.9552356, type: restaurant
Total page count: 3
Getting plcaes, city: San Jose, Californ

City        Type                     Place ID  \
0   San Francisco, California  restaurant  ChIJ0SMraI-AhYAREeJAvm2_yGM   
1   San Francisco, California  restaurant  ChIJN2S4EI2AhYAR9J4Qeh1U8Aw   
2   San Francisco, California  restaurant  ChIJURDKN2eAhYARN0AMzUEaiKo   
3   San Francisco, California  restaurant  ChIJ18LA7PSAhYARZCNy_dme7v8   
4   San Francisco, California  restaurant  ChIJIbtvA4qAhYARdnwerOi-f4I   
5   San Francisco, California  restaurant  ChIJO8CPF2GAhYARBKYOHZoWGCo   
6   San Francisco, California  restaurant  ChIJzWMEoXuAhYARnGNx8NeVTpY   
7   San Francisco, California  restaurant  ChIJP5PrLYSAhYARBcWhJXs55P4   
8   San Francisco, California  restaurant  ChIJiZyN7IeAhYARnCn-G6GH73Q   
9   San Francisco, California  restaurant  ChIJW52Si_OAhYAR-jAC1-iQh_c   
10  San Francisco, California  restaurant  ChIJSToWqn2AhYAR1CuH2iGMvuo   
11  San Francisco, California  restaurant  ChIJF-zbSmSAhYARvOafjdCRszQ   
12  San Francisco, California  restaurant  ChIJZduCKJGAhYARueNvSMvqsUI   
13  San Francisco, California  restaurant  ChIJNRipBPGAhYAR1qLFGG_Tby4   
14  San Francisco, California  restaurant  ChIJ6zMe3oWAhYARaZ33Z1BAMRo   
15  San Francisco, California  restaurant  ChIJf-NCNpeHhYARLeneHNw3WWs   
16  San Francisco, California  restaurant  ChIJn5G9Fzp-j4ARRs66QTw4XAY   
17  San Francisco, California  restaurant  ChIJNy8VmmGAhYARQF84_9twEc4   
18  San Francisco, California  restaurant  ChIJl7zcHY-AhYAR4rDx9C6-N4s   
19  San Francisco, California  restaurant  ChIJk2EFyIqAhYARXlKs2zO34Fg   
20  San Francisco, California  restaurant  ChIJ0SMraI-AhYAREeJAvm2_yGM   
21  San Francisco, California  restaurant  ChIJN2S4EI2AhYAR9J4Qeh1U8Aw   
22  San Francisco, California  restaurant  ChIJURDKN2eAhYARN0AMzUEaiKo   
23  San Francisco, California  restaurant  ChIJ18LA7PSAhYARZCNy_dme7v8   
24  San Francisco, California  restaurant  ChIJIbtvA4qAhYARdnwerOi-f4I   
25  San Francisco, California  restaurant  ChIJO8CPF2GAhYARBKYOHZoWGCo   
26  San Francisco, California  restaurant  ChIJzWMEoXuAhYARnGNx8NeVTpY   
27  San Francisco, California  restaurant  ChIJP5PrLYSAhYARBcWhJXs55P4   
28  San Francisco, California  restaurant  ChIJiZyN7IeAhYARnCn-G6GH73Q   
29  San Francisco, California  restaurant  ChIJW52Si_OAhYAR-jAC1-iQh_c   
..                        ...         ...                          ...   
30        Fremont, California  restaurant  ChIJ73JHc2KVj4ARKyOaBwHVbs0   
31        Fremont, California  restaurant  ChIJB711rZi_j4ARlXzFOlpnLKw   
32        Fremont, California  restaurant  ChIJ35izgpfAj4ARZfCyAnTJmQs   
33        Fremont, California  restaurant  ChIJR6xFQC7Hj4ARW-dri2UHkBg   
34        Fremont, California  restaurant  ChIJ_eDeeNLAj4ARGeit-lvegp8   
35        Fremont, California  restaurant  ChIJTzD2cqrAj4ARoueogJtfuFo   
36        Fremont, California  restaurant  ChIJT_b6T2rHj4ARvws68qu_jtQ   
37        Fremont, California  restaurant  ChIJoWiG71u_j4ARlU8LVrh_kwo   
38        Fremont, California  restaurant  ChIJI63XqoTAj4ARtW_a57xl0uM   
39        Fremont, California  restaurant  ChIJxSyD2py_j4AR8bBSWIwCowM   
40        Fremont, California  restaurant  ChIJ-67yulzHj4ARs917lkd5kjs   
41        Fremont, California  restaurant  ChIJjUT_xZHAj4AR5XNFHM665Dk   
42        Fremont, California  restaurant  ChIJpcto4h7Hj4ARj_BMAb5HcG4   
43        Fremont, California  restaurant  ChIJy17_9Fy_j4ARboo4_yljPOY   
44        Fremont, California  restaurant  ChIJBYDSuarAj4ARmgVRcrUygrs   
45        Fremont, California  restaurant  ChIJC4LlqpC_j4AR-YtYSlwFSUk   
46        Fremont, California  restaurant  ChIJ41ntlMaVj4ARndbC_tyjSNc   
47        Fremont, California  restaurant  ChIJRd8gnhPHj4ARfTgzNXBAh1A   
48        Fremont, California  restaurant  ChIJPa0Okpy_j4ARLJXmX04Hitc   
49        Fremont, California  restaurant  ChIJ4QAv8US_j4ARNmBNiyXpF64   
50        Fremont, California  restaurant  ChIJ73JHc2KVj4ARKyOaBwHVbs0   
51        Fremont, California  restaurant  ChIJB711rZi_j4ARlXzFOlpnLKw   
52        Fremont, California